In [2]:
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [8]:
X,y = make_classification(n_samples=10000, n_features=10, n_informative=3)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
print("Accuracy Score: {}".format(accuracy_score(y_test, y_pred))) 

Accuracy Score: 0.8425


# Bagging

In [25]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators= 500,
    max_samples=0.5,
    bootstrap=True,
    random_state = 42
)

In [31]:
bag.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.5,
                  n_estimators=500, random_state=42)

In [33]:
y_pred = bag.predict(X_test)

In [35]:
print("Accuracy Score Bagging: {}".format(accuracy_score(y_test, y_pred)))

Accuracy Score: 0.8905


In [41]:
# To check how many rows
bag.estimators_samples_[0].shape

(4000,)

In [47]:
# To check how many columns
bag.estimators_features_[0].shape

(10,)

In [106]:
bag_svm = BaggingClassifier(
    estimator=SVC(),
    n_estimators= 500,
    max_samples=0.5,
    bootstrap=True,
    random_state=42,
    n_jobs=-1
)

In [108]:
bag_svm.fit(X_train, y_train)
y_pred = bag_svm.predict(X_test)
print("Accuracy Score SVC: {}".format(accuracy_score(y_test, y_pred)))

Accuracy Score SVC: 0.839


# Pasting
## Same row sampling but without replacement

In [60]:
bag_pas = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators= 500,
    max_samples= 0.5,
    bootstrap = False,
    random_state= 42,
    n_jobs=1
)

In [62]:
bag_pas.fit(X_train, y_train)
y_pred = bag_pas.predict(X_test)
print("Accuracy Score Pasting: {}".format(accuracy_score(y_test, y_pred)))

Accuracy Score: 0.8875


## Random Subspaces
### Do not row sampling do Column sampling with or without replacement

In [68]:
bag_rand = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators= 500,
    max_samples= 1.0,
    bootstrap=False,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42,
    n_jobs=1
)

In [70]:
bag_rand.fit(X_train, y_train)
y_pred = bag_rand.predict(X_test)
print("Accuracy Score Random Subspaces: {}".format(accuracy_score(y_test, y_pred)))

Accuracy Score Random Subspaces: 0.884


## Random Patches
### Do both row and column sampling

In [73]:
bag_rand_patch = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42,
    n_jobs=1
)

In [77]:
bag_rand_patch.fit(X_train, y_train)
y_pred = bag_rand_patch.predict(X_test)
print("Accuracy Score Random Patches: {}".format(accuracy_score(y_test, y_pred)))

Accuracy Score Random Patches: 0.8815


# OOB Score - Out Of Bag Score
### It is statistically proven that when we perform random sampling 37% data is ignored while training, called OOB samples
### it's means that model trained only on 63% data so that we use OOB 

In [82]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    oob_score=True,
    random_state=42,
    n_jobs=1
)

In [84]:
bag.fit(X_train, y_train)
y_pred = bag.predict(X_test)
print("Accuracy Score OOB: {}".format(accuracy_score(y_test, y_pred)))

Accuracy Score OOB: 0.8925


# Bagging Tips
```
* Bagging generally gives better results than Pasting
* Good results come around the 25% to 50% row sampling mark
* Random patches and subspaces should be used while dealing with high dimensional data
* To find the correct hyperparameter values we can do GridSearchCV/RandomSearchCV
```

# Applying GridSearchCV

In [90]:
from sklearn.model_selection import GridSearchCV

In [92]:
parameters = {
    "n_estimators" : [50, 100, 500],
    "max_samples" : [0.25, 0.5, 0.4, 0.7, 1.0],
    "bootstrap" : [True, False],
    "max_features" : [0.25, 0.5, 0.4, 0.7, 1.0]
}

In [96]:
search = GridSearchCV(BaggingClassifier(), parameters, cv=5, verbose=2, n_jobs=-1)

In [100]:
search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=BaggingClassifier(),
             param_grid={'bootstrap': [True, False],
                         'max_features': [0.25, 0.5, 0.4, 0.7, 1.0],
                         'max_samples': [0.25, 0.5, 0.4, 0.7, 1.0],
                         'n_estimators': [50, 100, 500]})

In [102]:
search.best_params_

{'bootstrap': True,
 'max_features': 0.7,
 'max_samples': 0.7,
 'n_estimators': 500}

In [104]:
search.best_score_

0.892625

# Applying RandomizedSearchCV

In [110]:
from sklearn.model_selection import RandomizedSearchCV

In [114]:
search_random = RandomizedSearchCV(BaggingClassifier(), parameters, cv=5, verbose=2, n_jobs=-1)

In [118]:
search.fit(X_train, y_train)

KeyboardInterrupt: 